## groupBy()

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



groupBy  : 집계함수를 가지고 있는 GroupData 객체를 반환한다.  

GrouopData객체의 집계함수들을 사용해 grouping 된 데이터들의 집계결과를 저장하고 있는 DataFrame을 반환 받을 수 있다.

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
|     EPS| CCAAW|           20|Suburban| Non-public| Experimental|
|     IQN| CCAAW|           15|Suburban| Non-public| Experimental|
|     PGK| CCAAW|           21|Suburban| Non-public|     Standard|
|     UHU| CCAAW|           16|Suburban| Non-public| Experimental|
|     UWK| CCAAW|           19|Suburban| Non-public|     Standard|
|     A33| CIMBB|           19|   Urban| Non-public|     Standard|
|     EID| CIMBB|           21|   Urban| Non-public|     Standard|
|     HUJ| CIMBB|           17|   Urban| Non-public| Experimental|
|     PC6| CIMBB|           17|   Urban| Non-public|     Stand

+--------+-------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|
+--------+-------------+------------------+
|   Rural| Experimental|               211|
|    null|         null|              null|
|   Urban|     Standard|               631|
|Suburban|     Standard|               433|
|   Rural|     Standard|               327|
|Suburban| Experimental|               284|
|   Urban| Experimental|               275|
+--------+-------------+------------------+

지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.


+--------+-----+
|     loc|count|
+--------+-----+
|   Urban|   37|
|Suburban|   34|
|   Rural|   28|
+--------+-----+

지역별 교육타입별 학생 숫자와 평균을 구해보자. 단 학생 숫자가 300미만인 데이터는 제외한다.


+--------+-------------+------------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|avg(class_std_cnt)|
+--------+-------------+------------------+------------------+
|   Urban|     Standard|               631| 24.26923076923077|
|Suburban|     Standard|               433|             21.65|
|   Rural|     Standard|               327|           20.4375|
+--------+-------------+------------------+------------------+

컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자


+--------+-------------+-----+-----------------+
|     loc|teaching_type|total|              avg|
+--------+-------------+-----+-----------------+
|   Urban|     Standard|  631|24.26923076923077|
|Suburban|     Standard|  433|            21.65|
|   Rural|     Standard|  327|          20.4375|
+--------+-------------+-----+-----------------+



+--------+-------------+-----+-----------------+---+---+------------------+
|     loc|teaching_type|total|              avg|max|min|            stddev|
+--------+-------------+-----+-----------------+---+---+------------------+
|   Urban|     Standard|  631|24.26923076923077| 31| 17| 4.229020617662603|
|Suburban|     Standard|  433|            21.65| 27| 15|3.1668513334889994|
|   Rural|     Standard|  327|          20.4375| 24| 16|2.1899391163530857|
+--------+-------------+-----+-----------------+---+---+------------------+



In [ ]:
# 학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이를 구해보자
cdf.groupby(cdf.loc) \
   .agg(count('school')
        , sum('class_std_cnt').alias('tot')) \
   .where(cdf.loc.isNotNull()) \
   .select(max(col('tot')) - min(col('tot'))) \
   .show()



#### sql

In [ ]:
# dataFrame을 테이블로 등록
cdf.createOrReplaceTempView('class')


print('지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.')
spark.sql('''
    select loc, count(*) from class group by loc having loc is not null; 
''').show()


print('''지역내 교육타입별 학생 숫자와 평균을 구해보자. 
단  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.''')   
spark.sql('''
    select loc, teaching_type, sum(class_std_cnt), avg(class_std_cnt) from class group by loc, teaching_type; 
''').show()


print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자')
spark.sql('''
    select 
    loc, teaching_type, sum(class_std_cnt) as total, avg(class_std_cnt) as avg 
    from class group by loc, teaching_type; 
''').show()

## orderBy()

In [ ]:
print('반 학생 숫자를 기준으로 내림차순 정렬하라')
cdf.orderBy(cdf.class_std_cnt.desc()).show(3)

print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
cdf.orderBy(cdf.loc.asc(), cdf.school.desc()).show(3)

# 학생들 시키기
print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
cdf.orderBy(cdf.school_type.asc_nulls_first()).show(3)


#### sql

In [ ]:
print('반 학생 숫자를 기준으로 내림차순 정렬하라')
spark.sql('''
select * from class order by class_std_cnt desc
''').show(3)


print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
spark.sql('''
select * from class order by loc asc, school desc
''').show(3)


# 학생들 시키기
print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
spark.sql('''
select * from class order by school_type asc nulls first
''').show(3)